# 4장 단어 수준 임베딩
## 4.1 NPLM

Neural Probabilistic Language Model(NPLM)은 자연어 처리 분야에 임베딩 개념을 널리 퍼뜨리는 데 일조한 선구자적 모델로, 딥러닝의 아버지 요슈아 벤지오 연구 팀이 제안한 기법(Bengio et al., 2003)이다.


### 모델 기본 구조
NPLM은 '단어가 어떤 순서로 쓰였는가'에서 설명한 통계 기반의 전통적인 언어 모델의 한계를 극복하는 과정에서 탄생했다. Bengio et al.(2003)은 기존 언어 모델의 단점을 다음과 같이 설명했다.

- 학습 데이터에 존재하지 않는 n-gram이 포함된 문장이 나타날 확률 값을 0으로 부여한다. 물론 **백오프**나 **스무딩**으로 이런 문제를 일부 보완할 수 있지만 완전한 것은 아니다.
- 문장의 **장기 의존성(long-term dependency)**를 포착해내기 어렵다. 다시 말해 n-gram 모델의 n을 5 이상으로 길게 설정할 수 없다. n이 커질수록 그 등장 확률이 0인 단어 시퀀스가 기하급수적으로 늘어난다. 
- 단어/문장 간 유사도를 계산할 수 없다. 

### NPLM의 학습
학습 파라미터의 종류가 많고 그 크기도 큰 편이다. 

## 4.2 Word2Vec
2013년 구글 연구 팀이 발표한 기법으로 가장 널리 쓰이고 있는 단어 임베딩 모델이다. 두 개의 논문으로 나누어 발표했는데, 하나는 Skip-Gram과 CBOW라는 모델이 제안됐고 다른 하나는 이 두 모델을 근간으로 하되 네거티브 샘플링 등 학습 최적화 기법을 제안한 내용이 핵심 골자이다. 

### 모델 기본 구조
- CBOW는 주변에 있는 **문맥 단어**들을 가지고 **타깃 단어** 하나를 맞추는 과정에서 학습된다. 
- Skip-gram 모델은 타깃 단어를 가지고 주변 문맥 단어가 무엇일지 예측하는 과정에서 학습된다. 
- CBOW의 경우 입력, 출력 학습 데이터 쌍이 {문맥 단어 4개, 타깃 단어} 하나인 반면, Skip-gram의 학습 데이터는 {타깃 단어, 타깃 직전 두 번째 단어}, {타깃 단어, 타깃 직전 단어}, {타깃 단어, 타깃 다음 단어}, {타깃 단어, 타깃 다음 두 번째 단어} 이렇게 4개쌍이 된다. Skip-gram이 같은 말뭉치로도 더 많은 학습 데이터를 확보할 수 있어서 임베딩 품질이 CBOW보다 좋은 경향이 있다. 따라서 Skip-gram 모델을 중심으로 Word2Vec 기법을 설명한다. 

In [1]:
# 튜토리얼

#git pull origin master
#bash preprocess.sh dump-tokenized

#cd /notebooks/embedding
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt 

In [ ]:
# Word2Vec Skip-gram 모델 학습

corpus_fname = '/notebooks/embedding/data/tokenized/corpus_mecab.txt'
model_fname = '/notebooks/embedding/data/word-embbedings/word2vec/word2vec'

from genism.models import Word2Vec

corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r').readlines()]
model = Word2Vec(corpus, size=100, workers=4, sg=1)
model.save(model_fname)

In [ ]:
# Word2Vec Skip-gram 모델 학습 스크립트
#cd /notebooks/embedding
#python models/word_utils.py --method train_word2vec \
--input_path data/tokenized/corpus_mecab.txt \
--input_path data/word-embeddings/word2vec/word2vec

In [ ]:
# 코사인 유사도 상위 단어 목록 체크 코드

from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator("/notebooks/embedding/data/word-embbedings/word2vec/word2vec", method="word2vec", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

#### 코사인 유사도 유의점
코사인 유사도 값이 높다도 해서 두 단어가 유의 관계인 것은 아니다. 예를 들어 '덥다'와 '춥다'는 반의 관계이지만, '기온'이라는 속성을 매개로 강한 관련을 맺고 있기 때문에 코사인 유사도 값이 높다. **관련성**이 높은 단어를 출력한다는 의미로 이해하는 것이 좋다. 

## 4.3 FastText

- FastText(Bojanowski et al, 2017)는 페이스북에서 개발해 공개한 단어 임베딩 기법이다.
- 각 단어를 **문자 단위 n-gram**으로 표현한다. 이 밖의 내용은 Word2Vec과 같다.

### 4.3.1 모델 기본 구조
- 예를 들어 ``시나브로``라는 단어의 문자 단위 n-gram(n=3)은 다음과 같은 **5개의 문자 단위 n-gram 벡터**로, 단어의 임베딩은 이 다섯 개의 벡터의 합으로 표현한다. 
    - <시나, 시나브, 나브로, 브로>, <시나브로>
- **네거티브 샘플링 기법**: Word2Vec 방식 + 타깃 단어(t), 문맥 단어(c) 쌍을 학습할 때 타깃 단어(t)에 속한 문자 단위 n-gram 벡터(z)들을 모두 업데이트한다. 
- FastText 모델에서 **포지티브 샘플이 주어졌을 때** 포지티브 샘플일 확률(분수 형태의 공식)을 최대화하려면 분모를 최소화해야 한다. 분모를 최소화하기 위해서는 z들과 $v_{c}$ 간 내적 값을 높여야 한다. 벡터의 내적은 코사인 유사도와 비례한다. 따라서 내적 값의 상향은 타깃 단어(t)에 속하는 문자 단위 n-gram 벡터와 문맥 단어의 포지티브 샘플(c)에 해당하는 단어 벡터 간 유사도를 높여야 한다는 의미로 이해할 수 있다.
    - 예를 들어 ``시나브로``가 타깃 단어(t), ``쌓였다``가 문맥 단어의 포지티브 샘플(c)이라면 ``<시나``, ``시나브``,``나브로``, ``브로>``, ``<시나브로>`` 등 문자 n-gram 벡터(z)들 각각을 ``쌓였다``에 해당하는 단어 벡터($v_{c}$)와의 **유사도를 높인다**. 
- 또한 **네거티브 샘플이 주어졌을 때** 네거티브 샘플일 확률(분수 형태의 공식)을 최대화하려면 분자를 최대화해야 한다. 즉, 네거티브 샘플을 모델에 입력하면 모델은 이 데이터가 정말 네거티브 샘플이라고 잘 맞춰야 한다. 그러기 위해서는 분자를 최대화해야 하는데 z들과 $v_{c}$ 간 내적 값을 낮춰야 한다. 마찬가지로 벡터의 내적은 코사인 유사도와 비례한다. 따라서 내적 값의 하향은 타깃 단어(t)에 속하는 문자 단위 n-gram 벡터와 문맥 단어의 네거티브 샘플(c)에 해당하는 단어 벡터 간 유사도를 낮춰야한다는 의미로 이해할 수 있다. 
    - 예를 들어 ``시나브로``가 타깃 단어(t), ``컴퓨터``가 문맥 단어의 네거티브 샘플(c)이라면, ``<시나``, ``시나브``,``나브로``, ``브로>``, ``<시나브로>`` 등 문자 n-gram 벡터(z)들 각각을 ``컴퓨터``에 해당하는 단어 벡터($v_{c}$)와의 **유사도를 낮춘다**.
- **로그우도 함수**: FastText 모델이 최대화해야 할 로그우도 함수, 모델을 한 번 업데이트할 때 **1개의 포지티브 샘플($t_{p}, c_{p}$)과 k개의 네거티브 샘플($t_{n_{i}}, c_{n_{i}}$)을 학습한다**는 의미 

### 4.3.2 튜토리얼

In [2]:
# 데이터 다운로드 
#git pull origin master
#bash preprocess.sh dump-tokenized

In [3]:
# 데이터 합치기
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [5]:
# FastText Skip-gram 모델 학습
#mkdir -p data/word-embeddings/fasttexrt
#models/fastText/fasttext skipgram - input data/tokenized/corpus_mecab.txt -output data/word-embeddings/fasttext/fasttext

In [ ]:
# FastText Skip-gram 모델의 코사인 유사도 상위 단어 목록 체크
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator(
    vecs_text_fname = 'data/word-embeddings/fasttext/fasttext.vec'
    vecs_bin_fname = 'data/word-embeddings/fasttext/fasttext.bin',
    method='fasttext', dim=100, tokenizer_name='mecab'
)
model.most_similar('희망', topn=5)
model.most_similar('하였다', topn=5)

- FastText는 오타나 **미등록 단어**에도 강건하다(robust). 각 단어의 임베딩을 문자 단위 n-gram 벡터의 합으로 표현하기 때문이다. 
    - 예를 들어 미등록 단어 ``서울 특별시``에 대해서도 FastText 임베딩을 추정할 수 있다. ``서울특별시``는 ``서울`` 같은 문자 단위 n-gram을 포함하고 있다. ``서울``이 어휘 집합에 있다면 나머지 n-gram(울특, 특별 등)이 모두 미등록 단어라 할지라도 ``서울특별시``에 대한 임베딩을 추정할 수 있다. 

In [ ]:
model._is_in_vocabulary('서울특별시')
model.get_word_vector('서울특별시')
model.most_similar('서울특별시')

### 4.3.3 한글 자소와 FastText

In [ ]:
# 데이터 다운로드 
#git pull origin master
#bash preprocess.sh dump-tokenized

In [ ]:
# 데이터 합치기
#cd /notebooks/embedding 
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [ ]:
# 한글 자소분해 예시
from preprocess import jamo_sentence, get_tokenizer
tokenizer = get_tokenizer('mecab')
tokens = ' '.join(tokenizer.morphs('나는 학교에 간다'))
print(jamo_sentence(tokens))

In [ ]:
# 은전한닢으로 형태소 분석된 말뭉치를 자소 단위로 분해
#python preprocess/unsupervised_nlputils.py --preprocess_mode jamo \ 
    -- input_path /notebooks/embedding/data/tokenized/wiki_ko_mecab.txt \
    -- output_path /notebooks/embedding/data/tokenized/wiki_ko_mecab_jamo.txt 

In [ ]:
# 자소 단위 FastText Skip-gram 모델 학습
cd /notebooks/embedding
mkdir -p data/word-embeddings/fasttext-jamo
models/fastText/fasttext skipgram \
    -input data/tokenized/corpus_mecab_jamo.txt \
    -output data/word-embeddings/fasttext-jamo/fasttext-jamo 

In [ ]:
# FastText Skip-gram 모델의 유사어 상위 목록 체크
from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator(
    vecs_txt_fname= 'data/word-embeddings/fasttext-jamo/fasttext-jamo.vec'
    vecs_bin_fname = 'data/word-embeddings/fasttext-jamo/fasttext-jamo.bin',
    method='fasttext-jamo', dim=100, tokenizer_name='mecab')
model.most_similar('희망')

In [ ]:
# 미등록 단어에 대한 자소 단위 FastText 임베딩 체크
model._is_in_vocabulary('서울특별시')
model.get_word_vector('서울특별시')
model.most_similar('서울특별시', topn=5)

## 4.4 잠재 의미 분석 
- 잠재 의미 분석(Latent Semantic Analysis)이란 단어-문서 행렬이나 TF-IDF 행렬, 단어-문맥 행렬 같은 커다란 행렬에 차원 축소 방법의 일종인 **특이값 분해**를 수행해 데이터의 차원 수를 줄여 계산 효율성을 키우는 한편 행간에 숨어 있는 잠재 의미를 이끌어내기 위한 방법론이다.
- 단어-문서 행렬이나 단어-문맥 행렬 등에 특이값 분해를 시행한 뒤 그 결과로 도출되는 행 벡터들을 단어 임베딩으로 사용할 수 있다. 
- **행렬 분해 기반의 기법**: GloVe나 Swivel과 더불어 잠재 의미 분석도 이에 해당됨 

### 4.4.1 PPMI 행렬 
- 단어-문서 행렬, TF-IDF 행렬, 단어-문맥 행렬, **점별 상호 정보량**에 모두 잠재 의미 분석을 수행할 수 있다.
- PMI: 두 확률변수 사이의 상관성을 계량화한 지표, 자연어 처리 분야에서 PMI는 **두 단어의 등장이 독립을 가정했을 때 대비 얼마나 자주 같이 등장하는지**를 수치화한 것으로 이해할 수 있다. 
$$ PMI (A, B) = log \frac{P(A, B)}{P(A) \times P(B)} $$
    - 수식에서 로그 안 우변의 분자가 분모보다 작을 경우: PMI는 음수가 됨, A와 B 두 단어가 동시에 등장할 확률이 두 단어가 독립일 때보다 작을 때 발생함
    - **그러나 이러한 수치는 우리가 가진 말뭉치의 크기가 충분히 크지 않는 한 신뢰하기 어렵다.** 보통 말뭉치에서 단어 하나의 등장 확률은 0.001 이하로 작은 편이다. 예를 들어 단어 A, B 각각의 등장 확률이 0.0001로 같은데 PMI가 음수가 되려면 두 단어가 동시에 나타날 확률 P(A, B)는 0.000000001보다 작아야 한다. 두 단어가 동시에 나타난 경우는 10억 개 가운데 1개 꼴로 매우 적다는 뜻이다. 더구나 단어 A, B가 단 한 번도 같이 등장하지 않는 경우 $PMI(A, B) = log 0 = -\infty$ 가 된다.

따라서 자연어 처리 분야에서는 PMI 대신 **양의 점별 상호 정보량**(PPMI, Positive Pointwise Mutual Information)이란 지표를 사용한다. **PMI가 양수가 아닐 경우 그 값을 신뢰하기 어려워 0으로 치환해 무시한다**는 뜻이다. 수식은 아래와 같다. 

$$ PPMI(A, B) = max(PMI(A, B), 0) $$

**Shifted PMI**(SPMI)란 PMI에서 $log k$를 빼준 값이다. k는 임의의 양의 상수이다. Shifted PMI는 Word2Vec과 깊은 연관이 있다는 논문이 발표되기도 했다. 수식은 아래와 같다. 
$$ SPMI(A, B) =PMI(A, B) - logk $$

### 4.4.2 행렬 분해로 이해하는 잠재 의미 분석

### 4.4.3 행렬 분해로 이해하는 Word2Vec

-- 이 두 파트의 개념은 나중에 정리하기

### 4.4.4 튜토리얼
단어-문맥 행렬과 PPMI 행렬에 대한 LSA 임베딩을 각각 구축해보자. 데이터는 네이버 영화 말뭉치 사용 

In [6]:
# 형태소 분석된 데이터 다운로드
#git pull origin master
#bash preprocess.sh dump-tokenized

# 데이터 합치기

#cd /notebooks/embedding 
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [ ]:
# 단어-문맥 행렬을 활용한 LSA
from sklearn.decomposition import TruncatedSVD
from soynlp.vectorizer import sent_to_word_context_matrix

corpus_fname = '/notebooks/embedding/data/tokenized/for-lsa-mecab.txt'

corpus = [sent.replace('\n', '').strip() for sent in open(corpus_fname, 'r').readlines()]
input_matrix, idx2vocab = sent_to_word_contexts_matrix(
        corpus,
        windows=3,
        min_tf=10,
        dynamic_weight=True,
        verbose=True)

cooc_svd = TruncatedSVD(n_componetns=100)
cooc_vecs = cooc_svd.fit_transform(input_matrix)

In [ ]:
# PPMI 행렬을 활용한 LSA
from soynlp.word import pmi

ppmi_matrix, _, _ = pmi(input_matrix, min_pmi=0)
ppmi_svd = TruncatedSVD(n_components=100)
ppmi_vecs = ppmi_svd.fit_transform(input_matrix)

In [ ]:
# LSA 학습 스크립트
#pyton models/word-utils.py --method latent_semantic_analysis \
#    --input_path /notebooks/embedding/data/tokenized/for-lsa-mecab.txt
#    --output_path /notebooks/embedding/data/word-embeddings/lsa/lsa

In [ ]:
# 코사인 유사도 상위 단어 목록 체크 (단어-문맥 행렬 + LSA)
from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator('data/word-embeddings/lsa/lsa-cooc.vecs',
                              method='lsa', dim=100, tokenizer_name='mecab')
model.most_similar('희망', topn=5)

In [ ]:
# 코사인 유사도 상위 단어 목록 체크 (PPMI + LSA)
from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator('data/word-embeddings/lsa/lsa-pmi.vecs',
                              method='lsa', dim=100, tokenizer_name='mecab')
model.most_similar('희망', topn=5)

## 4.5 GloVe
- 스탠포드대학교연구팀에서 개발한 단어 임베딩 기법
- Word2Vec과 잠재 의미 분석, 두 기법의 단점을 극복하고자 했다.
- 잠재 의미 분석은 말뭉치 전체의 통계량을 모두 활용할 수 있지만, 그 결과물로 단어 간 유사도를 측정하기는 어렵다.
- 한편 Word2Vec은 단어 벡터 사이의 유사도를 측정하는 데는 LSA보다 유리하지만, 사용자가 지정한 윈도우 내의 로컬 문맥(local context)만 학습하기 때문에 말뭉치 전체의 통계 정보는 반영되기 어렵다는 단점을 지닌다. 

### 4.5.1 모델 기본 구조 
- 임베딩된 두 단어 벡터의 내적이 말뭉치 전체에서의 동시 등장 빈도의 로그 값이 되도록 **목적함수**를 정의했다.
- '임베딩된 단어 벡터 간 유사도 측정을 수월하게 하면서도 말뭉치 전체의 통계 정보를 좀 더 잘 반영해보자'가 GloVe가 지향하는 핵심 목표
- 수식을 보면 단어 i, j 각각에 해당하는 벡터 $U_{i}$, $V_{j}$ 사이의 **내적 값과 두 단어 동시 등장 빈도($A_{ij}$)의 로그 값 사이의 차이가 최소화될수록 학습 손실이 작아진다.**
- GloVe는 일단 학습 말뭉치를 대상으로 **단어-문맥 행렬** A를 만드는 것에서부터 학습을 시작한다. 어휘 집합 크기가 1만 개 정도 되는 말뭉치라면 요소 개수가 1만x1만=1억이나 되는 큰 행렬을 만들어야 한다. 이후 목적함수를 최소화하는 임베딩 벡터를 찾기 위해 행렬 분해를 수행한다. 처음에 행렬 U, V를 랜덤으로 초기화한 뒤 수식을 최소화하는 방향으로 U, V를 조금씩 업데이트해 나간다. 학습 손실이 더 줄지 않거나 정해진 스텝 수만큼 학습했을 경우 학습을 종료한다. 학습이 끝나면 U를 단어 임베딩으로 쓸 수 있다. 이밖에 $U + V^T$, $U$와 $V^T$를 이어 붙여 임베딩으로 사용하는 것도 가능하다.

### 4.5.2 튜토리얼

In [ ]:
# 데이터 다운로드
#git pull origin master
#bash preprocess.sh dump-tokenized 

In [ ]:
# 데이터 합치기
#cd /notebooks/embedding 
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [ ]:
# GloVe 학습
cd /notebooks/embedding
mkdir -p data/word-embeddings/glove
models/glove/build/vocab_count -min-count 5 -verbose 2 < data/tokenized/corpus_mecab.txt > data/word-embeddings/glove/glove.vocab
models/glove/build/cooccur -memory 10.0 -vocab-file data/word-embeddings/glove/glove.vocab -verbose 2 -window-size 15 < data/tokenized/corpus_mecab.txt > data/word-embeddings/glove/glove/cooc
models/glove/build/shuffle -memory 10.0 -verbose 2 < data/word-embeddings/glove/glove.cooc > data/word-embeddings/glove/glove.shuf
models/glove/build/glove -save-file data/word-embeddings/glove/glove.vecs
-threads 4 -input-file data/word-embeddings/glove/glove.shuf -x-max 10 -iter 15 -vector-size 100 -binary 2 -vocab-file data/word-embeddings/glove/glove.vocab -verbose 2

In [ ]:
# 코사인 유사도 상위 단어 목록 체크
from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator('data/word-embeddings/glove/glove.txt', method='glove', dim=100, tokenizer_name='mecab')
model.most_similar('희망', topn=5)

## 4.6 Swivel
- Swivel은 구글 연구 팀이 발표한 행렬 분해 기반의 단어 임베딩 기법
- PMI 행렬을 U와 V로 분해하고, 학습이 종료되면 U를 단어 임베딩으로 쓸 수 있다.

### 4.6.1 모델 기본 구조
- Swivel은 PMI 행렬을 분해한다는 점에서 단어-문맥 행렬을 분해하는 GloVe와 다르다.
- 타깃 단어 i에 대응하는 $U_{i}$ 벡터와 문맥 단어 j에 해당하는 $V_{j}$ 벡터의 내적이 두 단어의 PMI 값과 일치하도록 두 벡터를 조금씩 업데이트한다. 단어 i, j가 같이 자주 등장할수록 두 단어에 해당하는 벡터의 내적이 PMI 값과 일치하도록 더욱 강제한다.
- 두 단어가 한 번도 등장하지 않았을 때 PMI는 음의 무한대로 발산하기 때문에 Shazeer et al.(2016)은 이 같은 케이스에 대해 목적함수를 별도로 설정했다.
- 이 부분 수식을 직관적으로 이해하는 것이 어렵다. 

In [ ]:
# 데이터 다운로드
#git pull origin master
#bash preprocess.sh dump-tokenized 

In [ ]:
# 데이터 합치기
#cd /notebooks/embedding 
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt

In [ ]:
# Swivel 모델 학습
cd /notebooks/embedding
mkdir -p data/word-embeddings/swivel
models/swivel/fastprep --input data/tokenized/corpus_mecab.txt --output_dir data/word-embeddings/swivel/swivel.data
python models/swivel/swivel.py --input_base_path data/word-embeddings/swivel/swivel.data --output_base_path data/word-embeddings/swivel --dim 100 

In [ ]:
# Swivel 모델의 코사인 유사도 상위 목록 체크
from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator('data/word-embeddings/swivel/row_embedding.tsv',
                               method = 'swivel', dim=100, tokenizer_name='mecab')
model.most_similar('희망', topn=5)
                               

## 4.7 어떤 단어 임베딩을 사용할 것인가
- 자연어 단어 간 통사적, 의미론적 관계가 임베딩에 얼마나 잘 녹아 있는지 정량적으로 평가
- 크게 단어 유사도 평가(word similarity test)와 단어 유추 평가(word analogy test)가 있다.

In [ ]:
# 단어 임베딩 다운로드
#git pull origin master
#bash preprocess.sh dump-word-embeddings

### 4.7.2 단어 유사도 평가 
= 단어 유사도 평가: 사람이 평가한 점수와 단어 벡터 간 코사인 유사도 사이의 **상관관계**를 계산해 단어 임베딩의 품질을 평가하는 방법 

In [ ]:
# 단어 유사도 평가 데이터셋 다운로드
#wget https://github.com/dongjun-Lee/kor2vec/blob/master/test_dataset/kor_ws353.csv -P /notebooks/embedding/data/raw 

In [ ]:
# 평가 클래스 로드
from models.word_eval import WordEmbeddingEvaluator

model_name = 'word2vec'

if model_name == 'word2vec':
    model = WordEmbeddingEvaluator(vecs_txt_fname = 'data/word-embeddings/word2vec/word2vec',
                                   method='wrod2vec', dim=100, tokenizer_name='mecab')
elif model_name == 'fasttext':
    model = WordEmbeddingEvaluator(vecs_txt_fname = 'data/word-embeddings/fasttext/fasttext.vec',
                                   vecs_bin_fname = 'data/word-embeddings/fasttext/fasttext.bin',
                                   method='fasttext', dim=100, tokenizer_name='mecab')
elif model_name == 'glove':
    model = WordEmbeddingEvaluator(vecs_txt_fname = 'data/word-embeddings/glove/glove.txt',
                                   method='glove', dim=100, tokenizer_name='mecab')    
elif model_name == 'swivel':
    model = WordEmbeddingEvaluator(vecs_txt_fname = 'data/word-embeddings/swivel/swivel.tsv',
                                   method='swivel', dim=100, tokenizer_name='mecab')    
else:
    print('model name error!')

In [ ]:
# 단어 유사도 평가 수행 코드
model.word_sim_test('data/raw/kor_ws353.csv')